# Reinforcement Learning Taxi Driving

This notebook is the implementation of [this article](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/).

In [1]:
import gym
import random
from IPython.display import clear_output

## Playing with Environment

In [2]:
env = gym.make("Taxi-v3").env

In [3]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
print(f"Action Space: {env.action_space}")
print(f"State Space: {env.observation_space}")

Action Space: Discrete(6)
State Space: Discrete(500)


In [6]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## Without Reinforcement Learning

In [7]:
env.s = 328

epoches = 0
penalties, reward = 0,0

# for animation
frames = []

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if reward == -10:
        penalties += 1
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )
    epoches += 1
print(f"Epoch: {epoches}")
print(f"Penalties incurred: {penalties}")

Epoch: 4052
Penalties incurred: 1278


In [8]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
#print_frames(frames)

## With Reinforcement Learning Using Q-Learning

In [9]:
import numpy as np

In [10]:
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [11]:
q_table.shape

(500, 6)

### Training The Agent

In [12]:
# Hyperparameters
alpha = 0.1                     # Learning Rate
gamma = 0.6                     # Importance of long term rewards
epsilon = 0.1                   # The chance of random move rather than learned one.

In [13]:
# Data for Plotting
all_epochs = []
all_penalties = []

In [16]:
for i in range(1, 100_000):
    state = env.reset()                                     # Start at random state
    epochs, penalties, reward = 0,0,0                       # init variables
    done = False                       
    while not done:                                         # while the agent has not finished and epoch
        # Decide whether or not go random or use qtable
        if random.uniform(0,1) < epsilon: 
            action = env.action_space.sample()              # Explore the action space
        else:
            action = np.argmax(q_table[state])              # Use learnt q-values
        
        # Act and get respond from the environment
        next_state, reward, done, _ = env.step(action) 
        
        # Update Q-Value of the current state
        old_q_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        new_q_value = (1 - alpha) * old_q_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_q_value
        
        if reward == -10:
            penalties += 1
        
        state = next_state
        epochs += 1

In [17]:
q_table[328]

array([ -2.40443764,  -2.27325184,  -2.40611273,  -2.36030474,
       -10.60794877, -10.288425  ])

## Evaluate Performance of The Agent After Q-learning

In [18]:
total_epochs, total_penalties = 0, 0
episodes = 1000

for _ in range (episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0 
    
    done = False
    
    while not done:
        
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        if reward == -10:
            penalties += 1
            
        epochs += 1
        
    total_penalties += penalties
    total_epochs += epochs

In [19]:
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 1000 episodes:
Average timesteps per episode: 13.127
Average penalties per episode: 0.0


# Visualize The Taxi Driving

In [20]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [21]:
# for animation
frames = []

In [22]:
done = False
while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

In [23]:
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 10
State: 0
Action: 5
Reward: 20
